# Core Imports and Setup

In [1]:
import os
from pathlib import Path

import warnings
warnings.filterwarnings("ignore")

import logging
logging.getLogger("openff.toolkit").setLevel(logging.ERROR)

from openff import toolkit, evaluator

LICENSE: Could not open license file "oe_license.txt" in local directory
LICENSE: N.B. OE_LICENSE environment variable is not set
LICENSE: N.B. OE_DIR environment variable is not set
LICENSE: No product keys!
LICENSE: No product keys!
LICENSE: No product keys!
LICENSE: No product keys!


# 0) Registering Custom ThermoML Properties

In [2]:
from openff.units import unit
from openff.evaluator import properties
from openff.evaluator.datasets.thermoml import thermoml_property
from openff.evaluator.datasets import PhysicalProperty, PropertyPhase

@thermoml_property("Osmotic coefficient", supported_phases=PropertyPhase.Liquid)
class OsmoticCoefficient(PhysicalProperty):
    """A class representation of a osmotic coeff property"""

    @classmethod
    def default_unit(cls):
        return unit.dimensionless

custom_thermoml_props = [
    OsmoticCoefficient,
]

for custom_prop_cls in custom_thermoml_props:    
    setattr(properties, custom_prop_cls.__name__, custom_prop_cls)

# 2) Estimating Data Sets

## Loading data set and FF parameters

In [3]:
import logging
from collections import defaultdict
from enum import Enum
from os import path

import numpy as np
from openff.units import unit
from openmm import app

from openff.evaluator.forcefield import SmirnoffForceFieldSource
from openff.evaluator.attributes import UNDEFINED
from openff.evaluator.substances import Component, ExactAmount, MoleFraction, Substance
from openff.evaluator.utils import packmol
from openff.evaluator.workflow import Protocol, workflow_protocol
from openff.evaluator.workflow.attributes import InputAttribute, OutputAttribute
from openff.evaluator.protocols import coordinates, forcefield
from openff.evaluator.workflow import utils
from openff.evaluator.workflow import Workflow

In [4]:
force_field_path = "openff-1.0.0.offxml"
force_field_source = SmirnoffForceFieldSource.from_path(force_field_path)

# Define a protocol which will build some coordinates for a system.
build_coordinates = coordinates.BuildCoordinatesPackmol("build_coordinates")
build_coordinates.max_molecules = 1000
build_coordinates.mass_density = 1.0 * unit.gram / unit.millilitre
build_coordinates.substance = Substance.from_components("O", "CO")
build_coordinates.substance = utils.ProtocolPath("substance", "global")

# Define a protocol which will assign force field parameters to the system.
assign_parameters = forcefield.BuildSmirnoffSystem(f"assign_parameters")
assign_parameters.water_model = forcefield.TemplateBuildSystem.WaterModel.TIP3P
assign_parameters.force_field_path = "openff-1.0.0.offxml"

# Set the `coordinate_file_path` input of the `assign_parameters` protocol
# to the `coordinate_file_path` output of the `build_coordinates` protocol.
assign_parameters.coordinate_file_path = utils.ProtocolPath(
    "coordinate_file_path", build_coordinates.id
)

substances = [
    Substance.from_components("CO"),
    Substance.from_components("O"),
]



## Defining Calculation Schemas

In [6]:
from openff.evaluator.properties import Density,EnthalpyOfMixing
from openff.evaluator.client import RequestOptions


h_mix_schema = EnthalpyOfMixing.default_simulation_schema()
h_mix_schema.workflow_schema.final_value_source=utils.ProtocolPath("result", h_mix_schema)
h_mix_schema.workflow_schema.protocol_schemas = [build_coordinates.schema, assign_parameters.schema]

for substance in substances:
    # Define the metadata to make available to the workflow protocols.
    metadata = {"substance": substance}
    # Create the executable workflow object from its schema.
    h_mix_workflow = Workflow.from_schema(h_mix_schema.workflow_schema, metadata=metadata)

# # Create an options object which defines how the data set should be estimated.
# estimation_options = RequestOptions()

# # Specify that we only wish to use molecular simulation to estimate the data set.
# estimation_options.calculation_layers = ["SimulationLayer"]

# # Add our custom schemas, specifying that the should be used by the 'SimulationLayer'
# estimation_options.add_schema("SimulationLayer", "EnthalpyOfMixing", h_mix_schema)

# Add the individual protocol's schema representations to the workflow schema.

TypeError: sequence item 0: expected str instance, SimulationSchema found

## Launching a Server and Client

In [33]:
from openff.evaluator.backends import ComputeResources
from openff.evaluator.backends.dask import DaskLocalCluster
from openff.evaluator.server import EvaluatorServer
from openff.evaluator.client import EvaluatorClient
from openff.evaluator.client import ConnectionOptions

# define client to submit queries
port = 8117
evaluator_client = EvaluatorClient(ConnectionOptions(server_port=port))

# define available / preferred resources
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
resources = ComputeResources(
    number_of_threads=1,
    number_of_gpus=1,
    preferred_gpu_toolkit=ComputeResources.GPUToolkit.CUDA,
)

with DaskLocalCluster(number_of_workers=1, resources_per_worker=resources) as calculation_backend:
    # spin up server
    evaluator_server = EvaluatorServer(calculation_backend=calculation_backend, delete_working_files=False, port=port)
    evaluator_server.start(asynchronous=True)

    # estimate data set by submitting calculation schemas to newly-created server
    request, exception = evaluator_client.request_estimate(
        property_set=data_set,
        force_field_source=force_field_source,
        options=estimation_options,
    )

    # Wait for the results.
    results, exception = request.results(synchronous=True, polling_interval=30)
    assert exception is None

ValueError: Both a data set and force field source must be present to compute physical properties.

In [14]:
a = results.estimated_properties.json("estimated_data_set.json", format=True)

In [15]:
print(a)

{
  "@type": "openff.evaluator.datasets.datasets.PhysicalPropertyDataSet",
  "properties": [
    {
      "@type": "openff.evaluator.properties.density.Density",
      "gradients": [],
      "id": "773858b70c18451e817c70f611f8cd79",
      "phase": 2,
      "source": {
        "@type": "openff.evaluator.datasets.provenance.CalculationSource",
        "fidelity": "SimulationLayer",
        "provenance": "{\"protocol_schemas\": [{\"id\": \"773858b70c18451e817c70f611f8cd79|decorrelate_trajectory\", \"type\": \"DecorrelateTrajectory\", \"inputs\": {\".allow_merging\": true, \".time_series_statistics\": {\"full_path\": \"b749b50ecaae494394be40d2aaf9cef9|conditional_group/773858b70c18451e817c70f611f8cd79|average_density.time_series_statistics\", \"@type\": \"openff.evaluator.workflow.utils.ProtocolPath\"}, \".input_coordinate_file\": {\"full_path\": \"b749b50ecaae494394be40d2aaf9cef9|conditional_group/b749b50ecaae494394be40d2aaf9cef9|production_simulation_liquid.output_coordinate_file\", \"@ty